In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
#Reading the csv file
data = pd.read_csv('iris.csv')
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
species         150 non-null object
dtypes: float64(4), object(1)
memory usage: 7.0+ KB


In [15]:
#splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

In [16]:
train_set.head()

,sepal_length,sepal_width,petal_length,petal_width,species
22,4.6,3.6,1.0,0.2,setosa
15,5.7,4.4,1.5,0.4,setosa
65,6.7,3.1,4.4,1.4,versicolor
11,4.8,3.4,1.6,0.2,setosa
42,4.4,3.2,1.3,0.2,setosa


In [17]:
test_set.head()

,sepal_length,sepal_width,petal_length,petal_width,species
73,6.1,2.8,4.7,1.2,versicolor
18,5.7,3.8,1.7,0.3,setosa
118,7.7,2.6,6.9,2.3,virginica
78,6.0,2.9,4.5,1.5,versicolor
76,6.8,2.8,4.8,1.4,versicolor


In [18]:
#For training data
#Forming an array without the column "species"
Y_train = pd.DataFrame(train_set.species)
Y = np.array((Y_train == "virginica").astype(np.int))
m = len(Y)

X_train = train_set.drop(["species"], axis=1)
#Forming the X like [1,X1,X2,....,Xm]
X = np.c_[np.ones((m,1)),X_train]
dim = len(X[0])
#X_train_new

In [19]:
#For test data
#Forming an array without the columns "species"
Y_test = pd.DataFrame(test_set.species)
test_labels = np.array((Y_test == "virginica").astype(np.int))
m1 = len(test_labels)

X_test = test_set.drop(["species"], axis=1)
#Forming the X like [1,X1,X2,....,Xm1]
test_data = np.c_[np.ones((m1,1)),X_test]

In [20]:
#Sigmoid function : 1/(1+exp(-x))
def sigmoid(x):
    sigma = 1.0/(1 + np.exp(-x))
    return sigma

In [21]:
#Cost function where p is the predicted probability
#Cost : -((transpose(Y).log(p)) + (transpose(1-Y).log(1-p)))/m
def cost(y,p):
    m = len(y)
    cost = (-np.dot(y.transpose(), np.log(p)) - np.dot((1-y).transpose(), np.log(1-p)))/m
    return cost

In [22]:
#Computing the hessian matrix with predicted probabability 'p'
#Hessian matrix : (Second derivative of cost function)
#   derivative of gradient(MSE(theta) w.r.t theta) = (transpose(X).X.diagonal(p).diagonal(1-p))
def compute_hessian(x,p):
    m = len(x)
    hessian = (np.dot(x.transpose(), np.dot(x, np.dot(np.diag(p),np.diag(1-p)) ) ))/m
    inv_hessian = np.linalg.pinv(hessian)
    return inv_hessian

In [23]:
#Logistic Regression using Newton's method : (theta_new) = theta -(hessian*gradient(MSE(theta)))
#Here, gradient(MSE(theta)) = (transpose(X).(sigmoid(X.theta) - Y))/m --> (Predicted - actual)
def log_reg_newton_method(x,y,theta,iterations):
    m = len(y)
    theta_history = []
    cost_history = []
    for i in range(iterations):
        hypothesis = sigmoid(np.dot(x,theta)) #predicted probability
        eta = compute_hessian(x,hypothesis) #computing hessian 
        cost_history.append(cost(y,hypothesis))
        error = hypothesis - y
        gradient =(x.transpose().dot(error))/m
        theta = theta - np.dot(eta,gradient)
        theta_history.append(theta) 
    return cost_history,theta_history,theta

In [24]:
%%time
theta = np.zeros((dim,1))
cost_history,theta_history,theta_new = log_reg_newton_method(X,Y,theta,1000)

CPU times: user 140 ms, sys: 0 ns, total: 140 ms
Wall time: 139 ms


In [25]:
theta_new

array([[ 4313.25679168],
       [ -148.99944752],
       [-1425.01564118],
       [ -947.20464262],
       [ 3439.09015461]])

In [26]:
#The predicted probabilities will be the sigmoid
#Finding the predicted labels
predicted_probabilities = sigmoid(np.dot(test_data,theta_new))
predicted_labels = np.array((predicted_probabilities > 0.5).astype(np.int))

In [27]:
#Finding the accuracy score of the predicted test labels
from sklearn.metrics import accuracy_score

accuracy_score(test_labels,predicted_labels)

0.8